# Azure Cosmos DB for MongoDB (vCore)

>`Azure Cosmos DB for MongoDB in vCore` architecture makes it easy to create a database with full native MongoDB support. You can apply your MongoDB experience and continue to use your favorite MongoDB drivers, SDKs, and tools by pointing your application to the API for MongoDB (vCore) account's connection string.

This notebook goes over how to use the `AzureCosmosVCoreChatMessageHistory` class to store chat message history in a Azure Cosmos vCore database.


## Setup

The integration lives in the `langchain-community` package, so we need to install that.

```bash
pip install -U --quiet langchain-community
```

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability

In [1]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Usage

To use the storage you need to provide only 2 things:

1. Session Id - a unique identifier of the session, like user name, email, chat id etc.
2. Connection string - a string that specifies the database connection. Use a URL encoded connection string when connecting to a UNIX domain socket.

If you want to customize where the chat histories go, you can also pass:
1. *database_name* - name of the database to use
1. *collection_name* - collection to use within that database

In [2]:
from langchain_community.chat_message_histories import (
    AzureCosmosVCoreChatMessageHistory,
)

chat_message_history = AzureCosmosVCoreChatMessageHistory(
    session_id="test_session",
    connection_string="mongodb+srv://<user>:<password>@testdb.mongocluster.cosmos.azure.com/?tls=true&authMechanism=SCRAM-SHA-256&retrywrites=false&maxIdleTimeMS=120000",
    database_name="my_db",
    collection_name="chat_histories",
)

chat_message_history.add_user_message("Hello")
chat_message_history.add_ai_message("Hi")

/workspaces/langchain/libs/community/langchain_community/chat_message_histories/azure_cosmos_vcore.py:52: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  self.client: MongoClient = MongoClient(connection_string)


In [3]:
chat_message_history.messages

[HumanMessage(content='Hello'), AIMessage(content='Hi')]

## Chaining

We can easily combine this message history class with [LCEL Runnables](/docs/how_to/message_history)

To do this we will want to use OpenAI, so we need to install that.  You will also need to set the OPENAI_API_KEY environment variable to your OpenAI key.


In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import AzureChatOpenAI

In [5]:
import os

# "Set the OPENAI_API_VERSION environment variable with your Azure OpenAI API version."
os.environ["OPENAI_API_VERSION"] = "OPENAI_API_VERSION"
# "Set the OPENAI_API_KEY environment variable with your Azure OpenAI API key."
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
# "Set the AZURE_OPENAI_ENDPOINT environment variable with your Azure OpenAI endpoint."
os.environ["AZURE_OPENAI_ENDPOINT"] = "AZURE_OPENAI_ENDPOINT"

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | AzureChatOpenAI(
    azure_deployment="<Name_of_Azure_OpenAI_deployment_to_use>"
)

In [7]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: AzureCosmosVCoreChatMessageHistory(
        session_id=session_id,
        connection_string="mongodb+srv://<user>:<password>@testdb.mongocluster.cosmos.azure.com/?tls=true&authMechanism=SCRAM-SHA-256&retrywrites=false&maxIdleTimeMS=120000",
        database_name="my_db",
        collection_name="chat_histories",
    ),
    input_messages_key="question",
    history_messages_key="history",
)

In [8]:
# This is where we configure the session id
config = {"configurable": {"session_id": "<SESSION_ID>"}}

In [10]:
chain_with_history.invoke({"question": "Hi! I'm bob"}, config=config).content

/workspaces/langchain/libs/community/langchain_community/chat_message_histories/azure_cosmos_vcore.py:52: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  self.client: MongoClient = MongoClient(connection_string)


'Hello again, Bob! How can I help you today?'

In [11]:
chain_with_history.invoke({"question": "Whats my name"}, config=config).content

/workspaces/langchain/libs/community/langchain_community/chat_message_histories/azure_cosmos_vcore.py:52: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  self.client: MongoClient = MongoClient(connection_string)


'Your name is Bob. How can I assist you further?'